[View in Colaboratory](https://colab.research.google.com/github/battlerhythm/tensorflow/blob/master/6.2%20Dropout.ipynb)

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [0]:
# 과적합(Overfitting)으로 학습한 결과가 학습 데이터에는 매우 잘 맞지만,
# 학습 데이터에만 너무 꼭 맞춰져 있어서 그 외의 데이터에는 잘 맞지 않는 상황인 경우
# 드롭아웃(Dropout)을 통해 해결할 수 있습니다.
# 드롭아웃의 원리는 매우 간단한데, 학습시 전체 신경망 중 일부만을 사용하도록 하는 것입니다.
# 즉, 학습단계마다 일부 뉴런을 제거(사용하지 않도록)함으로써, 일부 특징이 특정 뉴런들에
# 고정되는 것을 막아 가중치의 균형을 잡도록 하여 과적합을 방지합니다.
# 다만, 학습 시 일부 뉴런을 학습시키지 않기 때문에 신경망이 충분히 학습되기까지의 시간은 조금 더 오래 걸리는 편입니다.

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
b1 = tf.Variable(tf.zeros([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
b2 = tf.Variable(tf.zeros([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
b3 = tf.Variable(tf.zeros([10]))
model = tf.matmul(L2, W3) + b3

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
# optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
# train_op = optimizer.minimize(cost)

In [9]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 학습데이터 전체를 한 바퀴 도는 것을 에포크(epoch)라고합니다.
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(30):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        # 학습 코드: keep_prob를 0.8로 넣어줍니다.
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.8})
        total_cost += cost_val
        
        
    print('Epoch: {epoch:04d} Avg. cost = {cost:.3f}'.format(epoch=(epoch + 1), cost=(total_cost / total_batch)))
    
print('최적화 완료')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# 예측코드: keep_prob를 1로 넣어줍니다.
print('정확도: {0:.4f}'.format(sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})))

Epoch: 0001 Avg. cost = 0.438
Epoch: 0002 Avg. cost = 0.166
Epoch: 0003 Avg. cost = 0.114
Epoch: 0004 Avg. cost = 0.090
Epoch: 0005 Avg. cost = 0.072
Epoch: 0006 Avg. cost = 0.058
Epoch: 0007 Avg. cost = 0.052
Epoch: 0008 Avg. cost = 0.047
Epoch: 0009 Avg. cost = 0.040
Epoch: 0010 Avg. cost = 0.037
Epoch: 0011 Avg. cost = 0.034
Epoch: 0012 Avg. cost = 0.032
Epoch: 0013 Avg. cost = 0.027
Epoch: 0014 Avg. cost = 0.026
Epoch: 0015 Avg. cost = 0.026
Epoch: 0016 Avg. cost = 0.023
Epoch: 0017 Avg. cost = 0.025
Epoch: 0018 Avg. cost = 0.023
Epoch: 0019 Avg. cost = 0.019
Epoch: 0020 Avg. cost = 0.021
Epoch: 0021 Avg. cost = 0.019
Epoch: 0022 Avg. cost = 0.018
Epoch: 0023 Avg. cost = 0.018
Epoch: 0024 Avg. cost = 0.018
Epoch: 0025 Avg. cost = 0.021
Epoch: 0026 Avg. cost = 0.016
Epoch: 0027 Avg. cost = 0.016
Epoch: 0028 Avg. cost = 0.014
Epoch: 0029 Avg. cost = 0.016
Epoch: 0030 Avg. cost = 0.016
최적화 완료
정확도: 0.9849
